In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lower_Grade_Glioma"
cohort = "GSE24072"

# Input paths
in_trait_dir = "../../input/GEO/Lower_Grade_Glioma"
in_cohort_dir = "../../input/GEO/Lower_Grade_Glioma/GSE24072"

# Output paths
out_data_file = "../../output/preprocess/Lower_Grade_Glioma/GSE24072.csv"
out_gene_data_file = "../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE24072.csv"
out_clinical_data_file = "../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE24072.csv"
json_path = "../../output/preprocess/Lower_Grade_Glioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"EXPRESSION OF VAV1 IN GLIOBLASTOMA MULTIFORME"
!Series_summary	"Background: Even though much progress has been made in the understanding of the molecular nature of glioma, the survival rates of patients affected of this tumour have not changed significantly during these years. Thus, a deeper understanding of this malignancy is still needed in order to predict its outcome and improve patient treatment. Here, we report that VAV1, a GDP/GTP exchange factor for Rho/Rac proteins with oncogenic potential that is involved in the regulation of cytoskeletal dynamics and cell migration."
!Series_summary	"Methodology/Principal Findings: VAV1 is overexpressed in 32 patients diagnosed with high-grade glioma. Such overexpression is linked to the parallel upregulation of a number of genes coding for proteins also involved in cell invasion- and migration-related processes. Unexpectedly, immunohistochemical experiments revealed that VAV1 is not expressed in glioma

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
# obtained using oligonucleotide microarrays (Affymetrix HU-133A).
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Looking at the Sample Characteristics Dictionary:
# - Index 2 has glioma grades which corresponds to our Lower_Grade_Glioma trait
# - Index 1 has age information
# - Index 0 has gender information
trait_row = 2
age_row = 1
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert glioma grade to binary: 0 for grade III, 1 for grade IV+"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Lower grade (grade III) = 0, higher grade (grade IV/V) = 1
    if 'grade III' in value.lower():
        return 0
    elif 'grade IV' in value.lower() or 'grade V' in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age value to continuous numeric value"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male"""
    if value is None or not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path, 
                              is_gene_available=is_gene_available, 
                              is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we extract clinical features
if trait_row is not None:
    # Create a dictionary to transform into a DataFrame 
    # using the Sample Characteristics Dictionary provided in the output
    sample_chars = {
        0: ['gender: male', 'gender: female'],
        1: ['age: 72', 'age: 70', 'age: 34', 'age: 54', 'age: 68', 'age: 30', 'age: 60', 'age: 73', 'age: 52', 'age: 65', 'age: 76', 'age: 51', 'age: 43', 'age: 67', 'age: 66', 'age: 69', 'age: 74', 'age: 36', 'age: 38', 'age: 63', 'age: 46', 'age: 55'],
        2: ['type: glioma grade III', 'type: glioma grade IV', 'type: glioma grade V']
    }
    
    # Convert the sample characteristics to a DataFrame format
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features using the function from the library
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Clinical Features Preview:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical features to CSV
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Clinical Features Preview:
{0: [nan, 72.0, 1.0], 1: [nan, 70.0, 0.0], 2: [nan, 34.0, nan], 3: [nan, 54.0, nan], 4: [nan, 68.0, nan], 5: [nan, 30.0, nan], 6: [nan, 60.0, nan], 7: [nan, 73.0, nan], 8: [nan, 52.0, nan], 9: [nan, 65.0, nan], 10: [nan, 76.0, nan], 11: [nan, 51.0, nan], 12: [nan, 43.0, nan], 13: [nan, 67.0, nan], 14: [nan, 66.0, nan], 15: [nan, 69.0, nan], 16: [nan, 74.0, nan], 17: [nan, 36.0, nan], 18: [nan, 38.0, nan], 19: [nan, 63.0, nan], 20: [nan, 46.0, nan], 21: [nan, 55.0, nan]}
Clinical features saved to ../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE24072.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"EXPRESSION OF VAV1 IN GLIOBLASTOMA MULTIFORME"
Line 1: !Series_geo_accession	"GSE24072"
Line 2: !Series_status	"Public on Sep 30 2010"
Line 3: !Series_submission_date	"Sep 10 2010"
Line 4: !Series_last_update_date	"Mar 26 2020"
Line 5: !Series_pubmed_id	"22864683"
Line 6: !Series_summary	"Background: Even though much progress has been made in the understanding of the molecular nature of glioma, the survival rates of patients affected of this tumour have not changed significantly during these years. Thus, a deeper understanding of this malignancy is still needed in order to predict its outcome and improve patient treatment. Here, we report that VAV1, a GDP/GTP exchange factor for Rho/Rac proteins with oncogenic potential that is involved in the regulation of cytoskeletal dynamics and cell migration."
Line 7: !Series_summary	"Methodology/Principal Findings: VAV1 is overexpressed in 32 patients diagnosed with high-grade glioma. Suc

### Step 4: Gene Identifier Review

In [5]:
# Examining gene identifiers from the gene expression data
# The gene IDs extracted from the matrix file are in the format like '1007_s_at', '1053_at'
# These appear to be Affymetrix probe IDs from a microarray, not standard human gene symbols

# Based on biomedical knowledge, Affymetrix IDs (with '_at' suffix) need to be mapped to 
# standard human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 735371 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /D

### Step 6: Gene Identifier Mapping

In [7]:
# Based on the gene annotation preview, we can see:
# - Gene expression data has probe IDs like '1007_s_at' as index
# - Gene annotation data has the same IDs in the 'ID' column
# - Gene symbols are stored in the 'Gene Symbol' column

# 1. Decide which columns to use for mapping
probe_id_column = 'ID'
gene_symbol_column = 'Gene Symbol'

print(f"Using {probe_id_column} for probe IDs and {gene_symbol_column} for gene symbols")

# 2. Get a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_id_column, gene_col=gene_symbol_column)
print(f"Created gene mapping with {len(gene_mapping)} rows")
print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert probe-level data to gene expression data
# This function will handle the many-to-many mapping relationships
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Successfully mapped gene expression data: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Normalize gene symbols to ensure consistent format
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Using ID for probe IDs and Gene Symbol for gene symbols
Created gene mapping with 21225 rows
Gene mapping preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
Successfully mapped gene expression data: (13830, 32)
First few gene symbols after mapping:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')
After normalization: (13542, 32)
First few normalized gene symbols:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS', 'AACS', 'AADAC', 'AAGAB',
       'AAK1', 'AAMDC'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE24072.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI Gene database
print("\nNormalizing gene symbols using NCBI Gene database...")
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data_normalized)} unique genes remain")
    gene_data_cleaned = gene_data_normalized
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_cleaned.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {str(e)}")
    print("Falling back to original gene data")
    gene_data_cleaned = gene_data.copy()

# 2. We need to recreate the clinical data from the original matrix file
print("\nExtracting clinical data directly from the matrix file...")
# Extract clinical data from the matrix file again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data
print("\nProcessing clinical data...")
# Create clinical features dataframe 
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Extracted clinical data with shape: {selected_clinical_df.shape}")
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")
    
    is_trait_available = True
else:
    selected_clinical_df = pd.DataFrame()
    is_trait_available = False
    print("No trait data available in clinical information.")

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure the sample IDs match between clinical and genetic data
        common_samples = list(set(selected_clinical_df.columns).intersection(set(gene_data_cleaned.columns)))
        
        if len(common_samples) == 0:
            print("Warning: No common samples between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_biased = True
        else:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            
            # Filter data to include only common samples
            clinical_subset = selected_clinical_df[common_samples]
            genetic_subset = gene_data_cleaned[common_samples]
            
            # Link the data
            linked_data = pd.concat([clinical_subset, genetic_subset], axis=0).T
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values
            print("\nHandling missing values in linked data...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait and demographic features are severely biased
            print("\nEvaluating trait and demographic feature bias...")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error during data linking: {str(e)}")
        linked_data = pd.DataFrame()
        is_biased = True
else:
    print("\nCannot create linked data: missing clinical or gene data")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final validation and saving
note = "This dataset contains gene expression data from astrocytoma cell lines with modified GFAP isoform expression. The trait represents different experimental conditions related to the GFAPδ/GFAPα ratio."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if len(linked_data) > 0 else True,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable and len(linked_data) > 0:
    print(f"\nSaving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully!")
else:
    print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")


Normalizing gene symbols using NCBI Gene database...
After normalization: 13542 unique genes remain


Normalized gene expression data saved to ../../output/preprocess/Lower_Grade_Glioma/gene_data/GSE24072.csv

Extracting clinical data directly from the matrix file...

Processing clinical data...
Extracted clinical data with shape: (3, 32)
Clinical data saved to: ../../output/preprocess/Lower_Grade_Glioma/clinical_data/GSE24072.csv

Linking clinical and genetic data...
Found 32 common samples between clinical and genetic data
Linked data shape: (32, 13545)

Handling missing values in linked data...
Data shape after handling missing values: (0, 2)

Evaluating trait and demographic feature bias...
Quartiles for 'Lower_Grade_Glioma':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Lower_Grade_Glioma' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Age' in this dataset is fine.

Abnormality detected in the cohort: GSE24072. Preprocessing failed.

Dataset n